In [1]:
import torch 
import torch.nn as nn 
import torchvision.models as models
from transformers import Wav2Vec2Model
import torchaudio
import cv2
import pandas as pd
import numpy as np
from PIL import Image
import os
import torch.optim as optim
from tqdm import tqdm
from transformers import Wav2Vec2Processor
import os
from pathlib import Path
from sklearn.model_selection import train_test_split
import cv2
import glob
import random
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Is CUDA available?", torch.cuda.is_available())
print("Device name:", torch.cuda.get_device_name(0))
print("Number of GPUs:", torch.cuda.device_count())

2025-04-29 11:21:47.709639: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745925707.986727      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745925708.071773      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Is CUDA available? False


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [2]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split, Dataset
from torchvision.models import resnet50, ResNet50_Weights


transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [3]:
processor= Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")



/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:61: FutureWarning: Loading a tokenizer inside Wav2Vec2Processor from a config that does not include a `tokenizer_class` attribute is deprecated and will be removed in v5. Please add `'tokenizer_class': 'Wav2Vec2CTCTokenizer'` attribute to either your `config.json` or `tokenizer_config.json` file to suppress this warning: 
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:311: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [4]:
from torch.utils.data import DataLoader, random_split, Dataset

In [5]:

def extract_frame(video_folder, output_folder):
    os.makedirs(output_folder, exist_ok =True)
    video_files=glob.glob((os.path.join(video_folder, "*.flv")))
    for video_path in video_files:
        video_name = os.path.basename(video_path).replace(".flv", "")
        output_path= os.path.join(output_folder, video_name)
        os.makedirs(output_path, exist_ok= True)
        cap =cv2.VideoCapture(video_path)
        frame_count=0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            if frame_count % 10 == 0:
               frame_path = os.path.join(output_path, f"frame_{frame_count:04d}.jpg")
               cv2.imwrite(frame_path, frame)
            frame_count +=1
        cap.release()
    print(f"Extracted frames to {output_folder}")
    !zip -r faces.zip /kaggle/working/faces

    

In [ ]:
extract_frame("/kaggle/input/crema-d-video", "/kaggle/working/faces")


In [6]:
face_root_folder= "/kaggle/input/faces-dataset/kaggle/working/faces"
audio_root_folder= "/kaggle/input/cremad/AudioWAV"
def find_audio_files_and_labels(folder_path):
    audio_files = []
    labels = []
    audio_extensions = (".wav", ".mp3", ".flac")
    emotion_map = {
        "HAP": 0,  # Happy
        "SAD": 1,  # Sad
        "ANG": 2,  # Angry
        "FEA": 3,  # Fear
        "DIS": 4,  # Disgust
        "NEU": 5   # Neutral
    }
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith(audio_extensions):
                audio_files.append(os.path.join(root, file))
                # Extract emotion from file name (e.g., 1001_DFA_ANG_XX.wav)
                parts = file.split("_")
                if len(parts) >= 3:
                    emotion_code = parts[2]
                    label = emotion_map.get(emotion_code, 5)  # Default to Neutral
                else:
                    label = 5
                labels.append(label)
    return audio_files, labels

In [9]:
# Find face image files and labels
def find_face_files_and_labels(folder_path):
    face_files = []
    labels = []
    image_extensions = (".jpg", ".jpeg", ".png")
    emotion_map = {
        "HAP": 0,  # Happy
        "SAD": 1,  # Sad
        "ANG": 2,  # Angry
        "FEA": 3,  # Fear
        "DIS": 4,  # Disgust
        "NEU": 5   # Neutral
    }
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith(image_extensions):
                face_files.append(os.path.join(root, file))
                # Extract emotion from parent folder (e.g., 1001_DFA_ANG_XX)
                folder_name = os.path.basename(root)
                parts = folder_name.split("_")
                if len(parts) >= 3:
                    emotion_code = parts[2]
                    label = emotion_map.get(emotion_code, 5)
                else:
                    label = 5
                labels.append(label)
    return face_files, labels

In [10]:
# Pair audio and face files
def pair_audio_face_files(audio_files, face_root_folder):
    paired_audio = []
    paired_face = []
    labels = []
    emotion_map = {
        "HAP": 0,  # Happy
        "SAD": 1,  # Sad
        "ANG": 2,  # Angry
        "FEA": 3,  # Fear
        "DIS": 4,  # Disgust
        "NEU": 5   # Neutral
    }
    for audio_file in audio_files:
        audio_name = os.path.basename(audio_file).replace(".wav", "")
        face_folder = os.path.join(face_root_folder, audio_name)
        face_files = glob.glob(os.path.join(face_folder, "*.jpg"))
        if face_files:  # If matching face frames exist
            # Use a random frame for variety
            paired_audio.append(audio_file)
            paired_face.append(random.choice(face_files))  # Random frame
            parts = audio_name.split("_")
            if len(parts) >= 3:
                emotion_code = parts[2]
                label = emotion_map.get(emotion_code, 5)
            else:
                label = 5
            labels.append(label)
    return paired_audio, paired_face, labels

In [11]:
audio_files, audio_labels = find_audio_files_and_labels(audio_root_folder)
print(f"Found {len(audio_files)} audio files")
face_files, face_labels = find_face_files_and_labels(face_root_folder)
print(f"Found {len(face_files)} face images")
paired_audio_files, paired_face_files, paired_labels =pair_audio_face_files(audio_files, face_root_folder)
print(f"Found {len(paired_audio_files)} paired audio-face samples")

Found 7442 audio files
Found 60367 face images
Found 7441 paired audio-face samples


In [12]:
# Train-test split
voice_train_files, voice_test_files, voice_train_labels, voice_test_labels = train_test_split(
    audio_files, audio_labels, test_size=0.2, random_state=42
)
face_train_files, face_test_files, face_train_labels, face_test_labels = train_test_split(
    face_files, face_labels, test_size=0.2, random_state=42
)
paired_train_audio, paired_test_audio, paired_train_face, paired_test_face, paired_train_labels, paired_test_labels = train_test_split(
    paired_audio_files, paired_face_files, paired_labels, test_size=0.2, random_state=42
)

In [14]:
class VoiceDataset(Dataset):
    def __init__(self, audio_files,labels,processor):
        self.audio_files = audio_files
        self.labels = labels
        self.processor = processor
    def __len__(self):
        return len(self.audio_files)
    def __getitem__(self, index):
      try: 
        waveform, sample_rate=  torchaudio.load(self.audio_files[index])
        if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0, keepdim=True)  # Average channels
        if sample_rate != 16000:
            resampler = torchaudio.transforms.Resample(orig_freq= sample_rate, new_freq= 16000)
            waveform= resampler(waveform)
        desired_length = 16000 * 4  

        if waveform.shape[1] < desired_length:
              pad_size = desired_length - waveform.shape[1]
              waveform = torch.nn.functional.pad(waveform, (0, pad_size))
        else:
               waveform = waveform[:, :desired_length]
        # Handle multi-channel audio
       
        
        inputs = self.processor(
                waveform.squeeze(0), sampling_rate=16000, return_tensors="pt", padding=True
            )
        
        out= {
                "input_values": inputs.input_values.squeeze(0),
                "label": torch.tensor(self.labels[index])
            }
        if hasattr(inputs, "attention_mask") and inputs.attention_mask is not None:
              out["attention_mask"] = inputs.attention_mask.squeeze(0)
        return out
            

           
        
      except Exception as e:
             print(f"Error processing {self.audio_files[index]}: {e}")
             raise e

In [15]:
class FaceDataset(Dataset):
    def __init__(self, face_files, labels, transform):
        self.face_files = face_files
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.face_files)

    def __getitem__(self, idx):
        try:
            image = Image.open(self.face_files[idx]).convert("RGB")
            image = self.transform(image)
            return {
                "image": image,
                "label": torch.tensor(self.labels[idx])
            }
        except Exception as e:
            print(f"Error processing {self.face_files[idx]}: {e}")
            return None

In [16]:
class PairedDataset(Dataset):
    def __init__(self, audio_files, face_files, labels, processor, transform):
        self.audio_files = audio_files
        self.face_files = face_files
        self.labels = labels
        self.processor = processor
        self.transform = transform

    def __len__(self):
        return len(self.audio_files)

    def __getitem__(self, idx):
        try:
            # Audio
            waveform, sample_rate = torchaudio.load(self.audio_files[idx])
            if sample_rate != 16000:
                waveform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)(waveform)
            if waveform.shape[0] > 1:
                waveform = waveform.mean(dim=0, keepdim=True)
            desired_length = 16000 * 4  

            if waveform.shape[1] < desired_length:
              pad_size = desired_length - waveform.shape[1]
              waveform = torch.nn.functional.pad(waveform, (0, pad_size))
            else:
               waveform = waveform[:, :desired_length]
            inputs = self.processor(
                waveform.squeeze(0), sampling_rate=16000, return_tensors="pt", padding=True
            )
            # Image
            image = Image.open(self.face_files[idx]).convert("RGB")
            image = self.transform(image)
            output= {
                "input_values": inputs.input_values.squeeze(0),
                # "attention_mask": inputs.attention_mask.squeeze(0) if inputs.attention_mask is not None else None,
                "image": image,
                "label": torch.tensor(self.labels[idx])
            }
            if hasattr(inputs, "attention_mask") and inputs.attention_mask is not None:
              output["attention_mask"] = inputs.attention_mask.squeeze(0)
            return output
        except Exception as e:
            print(f"Error processing {self.audio_files[idx]} or {self.face_files[idx]}: {e}")
            return None

In [17]:

# Create dataloaders
voice_train_dataset = VoiceDataset(voice_train_files, voice_train_labels, processor)
voice_test_dataset = VoiceDataset(voice_test_files, voice_test_labels, processor)
face_train_dataset = FaceDataset(face_train_files, face_train_labels, transform)
face_test_dataset = FaceDataset(face_test_files, face_test_labels, transform)
paired_train_dataset = PairedDataset(paired_train_audio, paired_train_face, paired_train_labels, processor, transform)
paired_test_dataset = PairedDataset(paired_test_audio, paired_test_face, paired_test_labels, processor, transform)

voice_train_dataloader = DataLoader(voice_train_dataset, batch_size=2, shuffle=True)
voice_test_dataloader = DataLoader(voice_test_dataset, batch_size=2, shuffle=False)
face_train_dataloader = DataLoader(face_train_dataset, batch_size=32, shuffle=True)
face_test_dataloader = DataLoader(face_test_dataset, batch_size=32, shuffle=False)
paired_train_dataloader = DataLoader(paired_train_dataset, batch_size=32, shuffle=True)
paired_test_dataloader = DataLoader(paired_test_dataset, batch_size=32, shuffle=False)

In [5]:
class FaceEmotionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.resnet= models.resnet50(weights=ResNet50_Weights.DEFAULT)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 6)
        for name, param in self.resnet.named_parameters():
            if "layer4" in name or "fc" in name:
                param.requires_grad = True
            else:
                param.requires_grad = False
        
    def forward(self, images):
        return self.resnet(images)

In [ ]:
help(nn.Module)

In [6]:
class VoiceEmotionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.wav2vec = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base")
        self.classifier= nn.Linear(self.wav2vec.config.hidden_size, 6)
        for name, param in self.wav2vec.named_parameters():
            if "encoder.layer.10" in name or "encoder.layer.11" in name:
                param.requires_grad = True  # UNFREEZE
            else:
                param.requires_grad = False  # KEEP FROZEN

    def forward(self, input_values, attention_mask=None):
        outputs = self.wav2vec(input_values = input_values, attention_mask=attention_mask)
        hidden_states= outputs.last_hidden_state
        pooled = hidden_states.mean(dim=1)
        return self.classifier(pooled)

In [7]:
#Combined Model Class
class CombinedEmotionModel(nn.Module):
    def __init__(self,voice_model,face_model):
        super().__init__()
        self.face_model= face_model
        self.voice_model= voice_model
        self.fusion= nn.Linear(12,6)
    def forward(self,*, input_values=None, attention_mask=None, images=None):
        
        voice_output = self.voice_model.forward(input_values=input_values, attention_mask=attention_mask)
        face_output = self.face_model(images)
        

        combined =torch.cat((voice_output, face_output), dim =1)
        return self.fusion(combined)


In [21]:
voice_model = VoiceEmotionModel().to(device)
face_model = FaceEmotionModel().to(device)


pytorch_model.bin:   0%|          | 0.00/380M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/380M [00:00<?, ?B/s]

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth

  0%|          | 0.00/97.8M [00:00<?, ?B/s]
 12%|█▏        | 11.9M/97.8M [00:00<00:00, 124MB/s]
 30%|██▉       | 28.9M/97.8M [00:00<00:00, 156MB/s]
 47%|████▋     | 46.1M/97.8M [00:00<00:00, 167MB/s]
 64%|██████▎   | 62.1M/97.8M [00:00<00:00, 165MB/s]
 80%|███████▉  | 78.0M/97.8M [00:00<00:00, 163MB/s]
100%|██████████| 97.8M/97.8M [00:00<00:00, 162MB/s]


In [28]:
# Define loss and optimizers
criterion = nn.CrossEntropyLoss()

face_optimizer = optim.Adam(face_model.parameters(), lr=1e-3)
combined_optimizer = optim.Adam(combined_model.parameters(), lr=1e-3)

In [34]:
voice_optimizer = optim.Adam(voice_model.parameters(), lr=1e-3)

In [35]:
# Train Voice Model (pre-training for better fusion)

voice_model.train()
for epoch in range(10):
    total_loss=0
    for batch in voice_train_dataloader:
        input_values = batch["input_values"].to(device)
        attention_mask = batch["attention_mask"].to(device) if "attention_mask" in batch else None
        labels = batch["label"].to(device)
        outputs = voice_model(input_values, attention_mask)
        loss = criterion(outputs, labels)

        voice_optimizer.zero_grad()
        loss.backward()
        voice_optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(voice_train_dataloader)
    print(f"Voice Epoch {epoch+1}, Loss: {avg_loss}")

Voice Epoch 1, Loss: 1.3702092512836384
Voice Epoch 2, Loss: 1.278234402956
Voice Epoch 3, Loss: 1.2240777473232272
Voice Epoch 4, Loss: 1.195454100706872
Voice Epoch 5, Loss: 1.176544658556896
Voice Epoch 6, Loss: 1.1626032928494767
Voice Epoch 7, Loss: 1.1467813611841458
Voice Epoch 8, Loss: 1.1393870631502525
Voice Epoch 9, Loss: 1.125921913915366
Voice Epoch 10, Loss: 1.1089906319137444


In [37]:
# Test Voice Model 
voice_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch in voice_test_dataloader:
        input_values = batch["input_values"].to(device)
        attention_mask = batch["attention_mask"].to(device) if "attention_mask" in batch else None
        labels = batch["label"].to(device)

        outputs = voice_model(input_values, attention_mask)
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

voice_accuracy = correct / total
print(f"Voice Model Test Accuracy: {voice_accuracy:.4f}")
torch.save(voice_model.state_dict(), "voice_emotion_model.pth")

Voice Model Test Accuracy: 0.4533


In [31]:
# Train Face Model 

face_model.train()
for epoch in range(3):
    total_loss = 0
    for batch in face_train_dataloader:
        images = batch["image"].to(device)
        labels = batch["label"].to(device)

        outputs = face_model(images)
        loss = criterion(outputs, labels)

        face_optimizer.zero_grad()
        loss.backward()
        face_optimizer.step() 
        total_loss += loss.item()

    print(f"Face Epoch {epoch+1}, Loss: {total_loss}")

Face Epoch 1, Loss: 1695.790233194828
Face Epoch 2, Loss: 1101.8616681396961
Face Epoch 3, Loss: 742.527104511857


In [33]:
# Test Face Model 
face_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch in face_test_dataloader:
        images = batch["image"].to(device)
        labels = batch["label"].to(device)

        outputs = face_model(images)
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

face_accuracy = correct / total
print(f"Face Model Test Accuracy: {face_accuracy:.4f}")
torch.save(face_model.state_dict(), "face_emotion_model.pth")

Face Model Test Accuracy: 0.7666


In [41]:
combined_model = CombinedEmotionModel(voice_model, face_model).to(device)

In [44]:
# Train Combined Model 

combined_model.train()
for epoch in range(3):
    loss_total = 0
    for batch in paired_train_dataloader:
        input_values = batch["input_values"].to(device)
        attention_mask = batch["attention_mask"].to(device) if "attention_mask" in batch else None
        images = batch["image"].to(device)
        labels = batch["label"].to(device)

        outputs = combined_model(input_values=input_values, attention_mask=attention_mask, images=images)
        loss = criterion(outputs, labels)

        combined_optimizer.zero_grad()
        loss.backward()
        combined_optimizer.step()
        loss_total += loss.item()
        
        

    print(f"Combined Epoch {epoch+1}, Loss: {loss_total}")


Combined Epoch 1, Loss: 163.02335065603256
Combined Epoch 2, Loss: 73.89563891291618
Combined Epoch 3, Loss: 40.71809824183583


In [ ]:
help(torch.Size)

In [47]:
# Test Combined Model
combined_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch in paired_test_dataloader:
        input_values = batch["input_values"].to(device)
        attention_mask = batch["attention_mask"].to(device) if "attention_mask" in batch else None
        images = batch["image"].to(device)
        labels = batch["label"].to(device)

        outputs = combined_model(input_values=input_values, attention_mask=attention_mask, images=images)
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

combined_accuracy = correct / total
print(f"Combined Model Test Accuracy: {combined_accuracy:.4f}")
torch.save(combined_model.state_dict(), "combined_emotion_model.pth")


Combined Model Test Accuracy: 0.7965
